**Instructions**: Run all cells and then print the variable 'out_string' to verify that tests passed.

In [1]:
import sys
import numpy as np

sys.path.append('../')
import utils
from time_step import Brasnett

This Jupyter Notebook contains the test of the B-TIM algorithm implementation. It goes through a set of possible values that test all the currently implemented processes in the B-TIM (Sept 2023). The names of these tests are listed in the table below.  

|Test Name|Precipitation|Existing snowpack|Below Freezing?|Densification|
|--|--|--|--|--|
|(Control) Freeze|No|No|No|N/A|
|(Control) Melt|No|No|Yes|N/A|
|Dry Cold|No|Yes|Yes|Cold|
|Dry Warm|No|Yes|Yes|Warm|
|Dry Melt|No|Yes|No|N/A|
|Snow Cold|Yes|No|Yes|Cold|
|Snow Warm|Yes|No|Yes|Warm|
|Rain no Snow|Yes|No|No|N/A|
|Rain on Snow|Yes|Yes|No|Warm|
|Snow on Snow Warm|Yes|Yes|Yes|Warm|
|Snow on Snow Cold|Yes|Yes|Yes|Cold|



In [2]:
class TestScenario:

    def __init__(self, scenarioname, Tlist, Plist, Dlist, rholist):

        self.name = scenarioname
        self.T = np.array(Tlist)
        self.P = np.array(Plist)
        self.depth = np.array(Dlist)
        self.density = np.array(rholist)

ControlFreeze = TestScenario('ControlCold', [-1, -1], [0.], [0.], [200.])
ControlMelt = TestScenario('ControlWarm', [1, 1], [0.], [0.], [200.])
DryCold = TestScenario('DryCold', [-2, -2], [0.], [0.04], [300.])
DryWarm = TestScenario('DryWarm', [-0.5, -0.5], [0.], [0.04], [300.])
DryMelt = TestScenario('DryMelt', [0.5, 0.5], [0.], [0.04], [300.])
SnowCold = TestScenario('SnowCold', [-2, -2], [0.001], [0.], [200.])
SnowWarm = TestScenario('SnowWarm', [-0.5, -0.5], [0.001], [0.], [200.])
RainNoSnow = TestScenario('RainNoSnow', [0.5, 0.5], [0.001], [0.], [200.])
RainOnSnow = TestScenario('RainOnSnow', [0.5, 0.5], [0.001], [0.04], [300.])
SnowOnSnowWarm = TestScenario('SnowOnSnowWarm', [-0.5, -0.5], [0.001], [0.04], [300.])
SnowOnSnowCold = TestScenario('SnowOnSnowCold', [-2, -2], [0.001], [0.04], [300.])

We test the implemented algorithm against a simple transcription of the model as described in Elias Chereque et al. (2023)

In [3]:
def test(T, P, depth, density):

    for i in range(2):
    
        precip = np.atleast_1d(0.8 * P[0])
        density = np.atleast_1d(np.maximum(200., np.minimum(550., density[0]))) #[kg/m^3]
    
        print('hour step init (T, P, rho, D):', T[i], precip, density[0], depth[0])

        if i == 0:
            gamma = np.minimum(5.5, np.maximum(0.1, (9.8e-3 * density) - 2.39))
            print('DENSITY, GAMMA, GAMMA/24:', density, gamma, gamma/24)
            
        if T[i] < 0:
            
            rho_new = np.atleast_1d(67.9 + 51.3*np.exp(np.atleast_1d(T[i]) / 2.6))  
            swefall = 1000 * precip/2
    
            swe_i = np.atleast_1d(depth[0]*density[0])

            if swe_i + swefall == 0:
                rho_star = np.atleast_1d(density[0])
                D_star = 0

            else:
                rho_star = (rho_new*swefall + np.atleast_1d(density[0])*swe_i)/(swe_i + swefall)
                #rho_star = (swefall + swe_i)/(depth[0] + swefall/rho_new)
        
                rho_star = np.atleast_1d(np.maximum(200., np.minimum(550., rho_star)))
                D_star = np.atleast_1d((swe_i+swefall)/rho_star)
            
        else:
            D_star = np.atleast_1d(depth[0])
            rho_star =  np.atleast_1d(density[0])
    
        print('New rho and D:', rho_star, D_star)

        delta_dm = -(T[0] - (-1)) * ((gamma/24)/rho_star) if T[0] > -1 else np.zeros_like(rho_star)
        delta_dm = np.atleast_1d(delta_dm)
        
        rhow = 1000 #[kg/m^3], density of water
        Cw = 4.18e3 #[J], specific heat of water
        rhoice = 917 #[kg/m^3], density of ice
        Lf = 0.334e6 #[J/kg], latent heat of fusion of water
        
        delta_dr = -(precip)*1000*Cw*(np.atleast_1d(T[0]))/(Lf*rho_star) if T[0] > 0 else np.zeros_like(rho_star)
        delta_dr = np.atleast_1d(delta_dr)
        
        print('Delta rain melt:', delta_dr)
        print('DENSITY, GAMMA:', rho_star, gamma/24)
        print('Delta melt:', delta_dm)
    
        D_star_star = np.atleast_1d(D_star + delta_dm/2 + delta_dr/2)
    
        D_star_star = D_star_star if D_star_star >= 0 else np.zeros_like(D_star_star)
    
        print('Dstarstar:', D_star_star)
        C1 = 2.
        C2 = 0.021
        C3 = 0.08
        B1 = 0.6
        
        Wmax = 700.
        W1 = 204.70
        W2 = 0.6730
        
        a = 2.778e-6
        
        if T[0] < -1:
            delta_rhod = C1*(B1 * (rho_star*D_star_star) * np.exp(C3*(T[0]-(-1))) * np.exp(-C2 * rho_star))/2 
        
        else:
            if D_star_star != 0:
                rhomax = Wmax - (W1/D_star_star)*(1-np.exp(-D_star_star/W2))
                if rhomax - rho_star > 0.1:
                    delta_rhod = (rhomax-rho_star)*(1-np.exp(-a*3600/2))
                else:
                    delta_rhod = 0.
            else:
                delta_rhod = 0.
    
        delta_rhod = np.atleast_1d(delta_rhod)
    
        print('Del rho, densification', delta_rhod)
        
        density = np.atleast_1d(rho_star + delta_rhod)
        
        density = np.where(density > 550., 550., density)
        density = np.where(density < 200., 200., density)
    
        depth = np.atleast_1d(D_star_star * (rho_star/density))
    
        print('Hour end rho, D:', density, depth)

    return depth, density


# Test: Control Freeze and Melt

In [4]:
T, P, depth, density = ControlFreeze.T, ControlFreeze.P, ControlFreeze.depth, ControlFreeze.density 

print('BRASNETT WITH DEBUGGING OUTPUT\n')
depth, density, swe = Brasnett([0,0], T, P, depth, density, debug_mode=True)

print('\nTEST WITH DEBUGGING OUTPUT\n')
test_depth, test_density = test(ControlFreeze.T, ControlFreeze.P, ControlFreeze.depth, ControlFreeze.density)

assert abs(depth - test_depth) < 1e-15
assert abs(density - test_density) < 1e-15

out_string = 'ControlFreeze TEST OK'

BRASNETT WITH DEBUGGING OUTPUT

hour step init (T, P, rho, D): [-1] [0.] [200.] [0.]
New rho and D: [200.] [0.]
Delta rain melt: [-0.]
DENSITY, GAMMA: [200.] [0.00416667]
Delta melt: [-0.]
Dstarstar: [0.]
Del rho, densification [0.]
Hour end rho, D: [200.] [0.]
hour step init (T, P, rho, D): [-1] [0.] [200.] [0.]
New rho and D: [200.] [0.]
Delta rain melt: [-0.]
DENSITY, GAMMA: [200.] [0.00416667]
Delta melt: [-0.]
Dstarstar: [0.]
Del rho, densification [0.]
Hour end rho, D: [200.] [0.]

TEST WITH DEBUGGING OUTPUT

hour step init (T, P, rho, D): -1 [0.] 200.0 0.0
DENSITY, GAMMA, GAMMA/24: [200.] [0.1] [0.00416667]
New rho and D: [200.] 0
Delta rain melt: [0.]
DENSITY, GAMMA: [200.] [0.00416667]
Delta melt: [0.]
Dstarstar: [0.]
Del rho, densification [0.]
Hour end rho, D: [200.] [0.]
hour step init (T, P, rho, D): -1 [0.] 200.0 0.0
New rho and D: [200.] 0
Delta rain melt: [0.]
DENSITY, GAMMA: [200.] [0.00416667]
Delta melt: [0.]
Dstarstar: [0.]
Del rho, densification [0.]
Hour end rho, 

In [5]:
T, P, depth, density = ControlMelt.T, ControlMelt.P, ControlMelt.depth, ControlMelt.density 

print('BRASNETT WITH DEBUGGING OUTPUT\n')
depth, density, swe = Brasnett([0,0], T, P, depth, density)

print('\nTEST WITH DEBUGGING OUTPUT\n')
test_depth, test_density = test(ControlMelt.T, ControlMelt.P, ControlMelt.depth, ControlMelt.density)

assert abs(depth - test_depth) < 1e-15
assert abs(density - test_density) < 1e-15

out_string += '\nControlMelt TEST OK'

BRASNETT WITH DEBUGGING OUTPUT


TEST WITH DEBUGGING OUTPUT

hour step init (T, P, rho, D): 1 [0.] 200.0 0.0
DENSITY, GAMMA, GAMMA/24: [200.] [0.1] [0.00416667]
New rho and D: [200.] [0.]
Delta rain melt: [-0.]
DENSITY, GAMMA: [200.] [0.00416667]
Delta melt: [-4.16666667e-05]
Dstarstar: [0.]
Del rho, densification [0.]
Hour end rho, D: [200.] [0.]
hour step init (T, P, rho, D): 1 [0.] 200.0 0.0
New rho and D: [200.] [0.]
Delta rain melt: [-0.]
DENSITY, GAMMA: [200.] [0.00416667]
Delta melt: [-4.16666667e-05]
Dstarstar: [0.]
Del rho, densification [0.]
Hour end rho, D: [200.] [0.]


# Test: Dry Cold

In [6]:
T, P, depth, density = DryCold.T, DryCold.P, DryCold.depth, DryCold.density 

print('BRASNETT WITH DEBUGGING OUTPUT\n')
depth, density, swe = Brasnett([0,0], T, P, depth, density)

print('\nTEST WITH DEBUGGING OUTPUT\n')
test_depth, test_density = test(DryCold.T, DryCold.P, DryCold.depth, DryCold.density)

assert abs(depth - test_depth) < 1e-15
assert abs(density - test_density) < 1e-15

out_string += '\nDryCold TEST OK'

BRASNETT WITH DEBUGGING OUTPUT


TEST WITH DEBUGGING OUTPUT

hour step init (T, P, rho, D): -2 [0.] 300.0 0.04
DENSITY, GAMMA, GAMMA/24: [300.] [0.55] [0.02291667]
New rho and D: [300.] [0.04]
Delta rain melt: [0.]
DENSITY, GAMMA: [300.] [0.02291667]
Delta melt: [0.]
Dstarstar: [0.04]
Del rho, densification [0.01220489]
Hour end rho, D: [300.01220489] [0.03999837]
hour step init (T, P, rho, D): -2 [0.] 300.0122048852877 0.03999837274816305
New rho and D: [300.01220489] [0.03999837]
Delta rain melt: [0.]
DENSITY, GAMMA: [300.01220489] [0.02291667]
Delta melt: [0.]
Dstarstar: [0.03999837]
Del rho, densification [0.01220176]
Hour end rho, D: [300.02440664] [0.03999675]


# Test: Dry Warm

In [7]:
T, P, depth, density = DryWarm.T, DryWarm.P, DryWarm.depth, DryWarm.density 

print('BRASNETT WITH DEBUGGING OUTPUT\n')
depth, density, swe = Brasnett([0,0], T, P, depth, density)

print('\nTEST WITH DEBUGGING OUTPUT\n')
test_depth, test_density = test(DryWarm.T, DryWarm.P, DryWarm.depth, DryWarm.density)

assert abs(depth - test_depth) < 1e-15
assert abs(density - test_density) < 1e-15

out_string += '\nDryWarm TEST OK'

BRASNETT WITH DEBUGGING OUTPUT


TEST WITH DEBUGGING OUTPUT

hour step init (T, P, rho, D): -0.5 [0.] 300.0 0.04
DENSITY, GAMMA, GAMMA/24: [300.] [0.55] [0.02291667]
New rho and D: [300.] [0.04]
Delta rain melt: [0.]
DENSITY, GAMMA: [300.] [0.02291667]
Delta melt: [-3.81944444e-05]
Dstarstar: [0.0399809]
Del rho, densification [0.52222449]
Hour end rho, D: [300.52222449] [0.03991143]
hour step init (T, P, rho, D): -0.5 [0.] 300.5222244876683 0.03991142702933609
New rho and D: [300.52222449] [0.03991143]
Delta rain melt: [0.]
DENSITY, GAMMA: [300.52222449] [0.02291667]
Delta melt: [-3.81280731e-05]
Dstarstar: [0.03989236]
Del rho, densification [0.51952374]
Hour end rho, D: [301.04174823] [0.03982352]


# Test: Dry Melt

In [8]:
T, P, depth, density = DryMelt.T, DryMelt.P, DryMelt.depth, DryMelt.density 

print('BRASNETT WITH DEBUGGING OUTPUT\n')
depth, density, swe = Brasnett([0,0], T, P, depth, density)

print('\nTEST WITH DEBUGGING OUTPUT\n')
test_depth, test_density = test(DryMelt.T, DryMelt.P, DryMelt.depth, DryMelt.density)

assert abs(depth - test_depth) < 1e-15
assert abs(density - test_density) < 1e-15

out_string += '\nDryMelt TEST OK'

BRASNETT WITH DEBUGGING OUTPUT


TEST WITH DEBUGGING OUTPUT

hour step init (T, P, rho, D): 0.5 [0.] 300.0 0.04
DENSITY, GAMMA, GAMMA/24: [300.] [0.55] [0.02291667]
New rho and D: [300.] [0.04]
Delta rain melt: [-0.]
DENSITY, GAMMA: [300.] [0.02291667]
Delta melt: [-0.00011458]
Dstarstar: [0.03994271]
Del rho, densification [0.5221831]
Hour end rho, D: [300.5221831] [0.0398733]
hour step init (T, P, rho, D): 0.5 [0.] 300.5221831040873 0.03987330444704541
New rho and D: [300.5221831] [0.0398733]
Delta rain melt: [-0.]
DENSITY, GAMMA: [300.5221831] [0.02291667]
Delta melt: [-0.00011438]
Dstarstar: [0.03981611]
Del rho, densification [0.51944132]
Hour end rho, D: [301.04162442] [0.03974741]


# Test: Snow Cold

In [9]:
SnowCold = TestScenario('SnowCold', [-2, -2], [0.001], [0.], [200.])

print('2:', SnowCold.depth)

print('\nTEST WITH DEBUGGING OUTPUT\n')
test_depth, test_density = test(SnowCold.T, SnowCold.P, SnowCold.depth, SnowCold.density)

T, P, depth, density = SnowCold.T, SnowCold.P, SnowCold.depth, SnowCold.density 

print('1:', depth)

print('BRASNETT WITH DEBUGGING OUTPUT\n')
depth, density, swe = Brasnett([0,0], T, P, depth, density)

assert abs(depth - test_depth) < 1e-15
assert abs(density - test_density) < 1e-15

out_string += '\nSnowCold TEST OK'

2: [0.]

TEST WITH DEBUGGING OUTPUT

hour step init (T, P, rho, D): -2 [0.0008] 200.0 0.0
DENSITY, GAMMA, GAMMA/24: [200.] [0.1] [0.00416667]
New rho and D: [200.] [0.002]
Delta rain melt: [0.]
DENSITY, GAMMA: [200.] [0.00416667]
Delta melt: [0.]
Dstarstar: [0.002]
Del rho, densification [0.00332224]
Hour end rho, D: [200.00332224] [0.00199997]
hour step init (T, P, rho, D): -2 [0.0008] 200.00332223890075 0.001999966778162847
New rho and D: [200.] [0.004]
Delta rain melt: [0.]
DENSITY, GAMMA: [200.] [0.00416667]
Delta melt: [0.]
Dstarstar: [0.004]
Del rho, densification [0.00664448]
Hour end rho, D: [200.00664448] [0.00399987]
1: [0.]
BRASNETT WITH DEBUGGING OUTPUT



# Test: Snow Warm

In [10]:
print('\nTEST WITH DEBUGGING OUTPUT\n')
test_depth, test_density = test(SnowWarm.T, SnowWarm.P, SnowWarm.depth, SnowWarm.density)

T, P, depth, density = SnowWarm.T, SnowWarm.P, SnowWarm.depth, SnowWarm.density 

print('BRASNETT WITH DEBUGGING OUTPUT\n')
depth, density, swe = Brasnett([0,0], T, P, depth, density)

assert abs(depth - test_depth) < 1e-15
assert abs(density - test_density) < 1e-15

out_string += '\nSnowWarm TEST OK'


TEST WITH DEBUGGING OUTPUT

hour step init (T, P, rho, D): -0.5 [0.0008] 200.0 0.0
DENSITY, GAMMA, GAMMA/24: [200.] [0.1] [0.00416667]
New rho and D: [200.] [0.002]
Delta rain melt: [0.]
DENSITY, GAMMA: [200.] [0.00416667]
Delta melt: [-1.04166667e-05]
Dstarstar: [0.00199479]
Del rho, densification [0.97907783]
Hour end rho, D: [200.97907783] [0.00198507]
hour step init (T, P, rho, D): -0.5 [0.0008] 200.9790778342193 0.0019850739571131894
New rho and D: [200.] [0.00399479]
Delta rain melt: [0.]
DENSITY, GAMMA: [200.] [0.00416667]
Delta melt: [-1.04166667e-05]
Dstarstar: [0.00398958]
Del rho, densification [0.98131959]
Hour end rho, D: [200.98131959] [0.0039701]
BRASNETT WITH DEBUGGING OUTPUT



# Test: Rain No Snow

In [11]:
print('\nTEST WITH DEBUGGING OUTPUT\n')
test_depth, test_density = test(RainNoSnow.T, RainNoSnow.P, RainNoSnow.depth, RainNoSnow.density)

T, P, depth, density = RainNoSnow.T, RainNoSnow.P, RainNoSnow.depth, RainNoSnow.density 

print('BRASNETT WITH DEBUGGING OUTPUT\n')
depth, density, swe = Brasnett([0,0], T, P, depth, density)

assert abs(depth - test_depth) < 1e-15
assert abs(density - test_density) < 1e-15

out_string += '\nRainNoSnow TEST OK'


TEST WITH DEBUGGING OUTPUT

hour step init (T, P, rho, D): 0.5 [0.0008] 200.0 0.0
DENSITY, GAMMA, GAMMA/24: [200.] [0.1] [0.00416667]
New rho and D: [200.] [0.]
Delta rain melt: [-2.50299401e-05]
DENSITY, GAMMA: [200.] [0.00416667]
Delta melt: [-3.125e-05]
Dstarstar: [0.]
Del rho, densification [0.]
Hour end rho, D: [200.] [0.]
hour step init (T, P, rho, D): 0.5 [0.0008] 200.0 0.0
New rho and D: [200.] [0.]
Delta rain melt: [-2.50299401e-05]
DENSITY, GAMMA: [200.] [0.00416667]
Delta melt: [-3.125e-05]
Dstarstar: [0.]
Del rho, densification [0.]
Hour end rho, D: [200.] [0.]
BRASNETT WITH DEBUGGING OUTPUT



# Test: Rain On Snow

In [12]:
print('\nTEST WITH DEBUGGING OUTPUT\n')
test_depth, test_density = test(RainOnSnow.T, RainOnSnow.P, RainOnSnow.depth, RainOnSnow.density)

T, P, depth, density = RainOnSnow.T, RainOnSnow.P, RainOnSnow.depth, RainOnSnow.density 

print('BRASNETT WITH DEBUGGING OUTPUT\n')
depth, density, swe = Brasnett([0,0], T, P, depth, density)

assert abs(depth - test_depth) < 1e-15
assert abs(density - test_density) < 1e-15

out_string += '\nRainOnSnow TEST OK'


TEST WITH DEBUGGING OUTPUT

hour step init (T, P, rho, D): 0.5 [0.0008] 300.0 0.04
DENSITY, GAMMA, GAMMA/24: [300.] [0.55] [0.02291667]
New rho and D: [300.] [0.04]
Delta rain melt: [-1.66866267e-05]
DENSITY, GAMMA: [300.] [0.02291667]
Delta melt: [-0.00011458]
Dstarstar: [0.03993437]
Del rho, densification [0.52217406]
Hour end rho, D: [300.52217406] [0.03986498]
hour step init (T, P, rho, D): 0.5 [0.0008] 300.5221740639216 0.03986497683009504
New rho and D: [300.52217406] [0.03986498]
Delta rain melt: [-1.66576328e-05]
DENSITY, GAMMA: [300.52217406] [0.02291667]
Delta melt: [-0.00011438]
Dstarstar: [0.03979946]
Del rho, densification [0.51942331]
Hour end rho, D: [301.04159738] [0.03973079]
BRASNETT WITH DEBUGGING OUTPUT



# Test: Snow on Snow Warm

In [13]:
print('\nTEST WITH DEBUGGING OUTPUT\n')
test_depth, test_density = test(SnowOnSnowWarm.T, SnowOnSnowWarm.P, SnowOnSnowWarm.depth, SnowOnSnowWarm.density)

T, P, depth, density = SnowOnSnowWarm.T, SnowOnSnowWarm.P, SnowOnSnowWarm.depth, SnowOnSnowWarm.density 

print('BRASNETT WITH DEBUGGING OUTPUT\n')
depth, density, swe = Brasnett([0,0], T, P, depth, density)

assert abs(depth - test_depth) < 1e-15
assert abs(density - test_density) < 1e-15

out_string += '\nSnowOnSnowWarm TEST OK'


TEST WITH DEBUGGING OUTPUT

hour step init (T, P, rho, D): -0.5 [0.0008] 300.0 0.04
DENSITY, GAMMA, GAMMA/24: [300.] [0.55] [0.02291667]
New rho and D: [293.87823281] [0.04219435]
Delta rain melt: [0.]
DENSITY, GAMMA: [293.87823281] [0.02291667]
Delta melt: [-3.89900716e-05]
Dstarstar: [0.04217485]
Del rho, densification [0.55513389]
Hour end rho, D: [294.4333667] [0.04209533]
hour step init (T, P, rho, D): -0.5 [0.0008] 294.43336670334014 0.0420953337324072
New rho and D: [288.67428432] [0.04432078]
Delta rain melt: [0.]
DENSITY, GAMMA: [288.67428432] [0.02291667]
Delta melt: [-3.96929479e-05]
Dstarstar: [0.04430094]
Del rho, densification [0.58338695]
Hour end rho, D: [289.25767126] [0.04421159]
BRASNETT WITH DEBUGGING OUTPUT



# Test: Snow on Snow Cold

In [14]:
print('\nTEST WITH DEBUGGING OUTPUT\n')
test_depth, test_density = test(SnowOnSnowCold.T, SnowOnSnowCold.P, SnowOnSnowCold.depth, SnowOnSnowCold.density)

T, P, depth, density = SnowOnSnowCold.T, SnowOnSnowCold.P, SnowOnSnowCold.depth, SnowOnSnowCold.density 

print('BRASNETT WITH DEBUGGING OUTPUT\n')
depth, density, swe = Brasnett([0,0], T, P, depth, density)

assert abs(depth - test_depth) < 1e-15
assert abs(density - test_density) < 1e-15

out_string += '\nSnowOnSnowCold TEST OK'


TEST WITH DEBUGGING OUTPUT

hour step init (T, P, rho, D): -2 [0.0008] 300.0 0.04
DENSITY, GAMMA, GAMMA/24: [300.] [0.55] [0.02291667]
New rho and D: [293.27970479] [0.04228046]
Delta rain melt: [0.]
DENSITY, GAMMA: [293.27970479] [0.02291667]
Delta melt: [0.]
Dstarstar: [0.04228046]
Del rho, densification [0.01452327]
Hour end rho, D: [293.29422807] [0.04227836]
hour step init (T, P, rho, D): -2 [0.0008] 293.2942280668712 0.04227836354547283
New rho and D: [286.99349746] [0.04460031]
Delta rain melt: [0.]
DENSITY, GAMMA: [286.99349746] [0.02291667]
Delta melt: [0.]
Dstarstar: [0.04460031]
Del rho, densification [0.0171074]
Hour end rho, D: [287.01060486] [0.04459766]
BRASNETT WITH DEBUGGING OUTPUT



## There should be 11 lines (one for each test) printed below with "TEST OK".

In [15]:
print(out_string)

ControlFreeze TEST OK
ControlMelt TEST OK
DryCold TEST OK
DryWarm TEST OK
DryMelt TEST OK
SnowCold TEST OK
SnowWarm TEST OK
RainNoSnow TEST OK
RainOnSnow TEST OK
SnowOnSnowWarm TEST OK
SnowOnSnowCold TEST OK
